# Ensemble of Modified Base Model : Model 6
(Dropout 25%, 1 extra CNN Layer)

## Approach
- Create a List of Base Models
- Take all the Preprocessed Images from each Class
- Split Training data into Train and Validation with 0.7 traim-test ratio
- Perform this splitting for each model in a loop with a different random state so that shuffling of images is different for each model.
- Perform Augmentation on each class of training data splitted above 
- Train the models on augmented data validating on 30% validation data
- Once all models in list are trained, use max voting to find predction of validation image set

Before running make sure there is no augmented & test folder in the directory of notebook since it is created in this file itself. If it exists delete them. A caveat here is that if you wish to run Model_1_2_3_4.ipynb file again, you need to run Preprocessing Notebook again to form required folders.

Also, if required change the path seperator to run code smoothly on windows

In [17]:
# Importing the Libraries
# For missing libraries
# pip install -U package_name --user

import keras
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from functools import partial
from sklearn.utils import shuffle
from skimage import io
import os
import cv2
from tqdm import tqdm
from glob import glob
import albumentations
from albumentations import ShiftScaleRotate, HorizontalFlip, VerticalFlip, RandomBrightnessContrast
import warnings
import shutil
warnings.filterwarnings("ignore")

In [18]:
# Defining Augmentation functions
# Shifting, Scaling & Rotations
def shiftscalerotate(images, save_path,
                     augment = True,
                     resize = [256,256],
                     random=1.0,
                     rotation=0,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     always=True,
                     call=1):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]

        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)

        if augment ==True:
            aug = ShiftScaleRotate(p=random, rotate_limit=(-30,30), interpolation=interpolation,
                                  always_apply=always)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1] # save original image and the augmented image
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x] 
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                aug_ext = '_ssr'*call
                tmp_img_name = f"{image_name}{aug_ext}.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
# Horizontal Flipping
def horizontalflip(images, save_path,
                   augment = True,
                   resize = [256,256],
                   random=1.0,
                   always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image and the masks 
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = HorizontalFlip(always_apply=always, p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1] # make sure only x1. If x is given in 2nd augmentation onwards, then duplicate images will be saved
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_hf.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
#vertical flipping
def verticalflip(images,
                 save_path,
                 augment = True,
                 resize=[256,256],
                 random=1.0,
                 always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = VerticalFlip(always_apply=always, p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1]
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_vf.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1
def brightness_contrast(images,
                        save_path,
                        augment = True,
                        resize=[256,256],
                        random=1.0,
                        brightness=0.2, 
                        contrast=0.2,
                        by_max=False,
                        always=True):
    H, W = resize
    for x in tqdm(images, total= len(images)):
        # First, we have to extract the image name, image extention.
        name = x.split("/")[-1].split(".")
        image_name = name[0]
        image_extn = name[1]
        # Now that we have the names, we have to read the image
        x = cv2.imread(x, cv2.IMREAD_COLOR)
        if augment ==True:
            aug = RandomBrightnessContrast(brightness_limit=brightness,
                                           contrast_limit=contrast,
                                           brightness_by_max=by_max,
                                           always_apply=always,
                                           p=random)
            augmented = aug(image=x)
            x1 = augmented["image"]
            save_images = [x, x1]
        # If the augment paramter is set to False, The function only saves the original image and mask to the defined save_path
        else:
            save_images = [x]
        idx = 0
        for i in save_images:
            i = cv2.resize(i, (W,H))
            if idx == 0:
                tmp_img_name = f"{image_name}.{image_extn}"
            else:
                tmp_img_name = f"{image_name}_bc.{image_extn}"
            image_path = os.path.join(save_path, tmp_img_name)
            cv2.imwrite(image_path, i)
            idx +=1

In [19]:
# Define ensemble of 10 models
n = 10
ensemble = [0]* n
def shuffling_tt(X, rs):
    X= shuffle(X, random_state=rs)
    return X

In [20]:
# Create & Complie Ensemble
DefaultConv2D = partial(keras.layers.Conv2D,
                        kernel_size=3, activation='relu',
                        padding="SAME")
for k in range(n):
    ensemble[k] = keras.models.Sequential([
                    DefaultConv2D(filters=16, input_shape=[128, 128, 3]),
                    keras.layers.AveragePooling2D(pool_size=2),
                    DefaultConv2D(filters=32),
                    keras.layers.MaxPooling2D(pool_size=2),
                    DefaultConv2D(filters=64),
                    keras.layers.MaxPooling2D(pool_size=2),
                    DefaultConv2D(filters=128),
                    keras.layers.MaxPooling2D(pool_size=2),
                    keras.layers.Dropout(0.25),
                    keras.layers.Flatten(),
                    keras.layers.Dense(units=7, activation='softmax'),
                    ])
    ensemble[k].compile(loss='categorical_crossentropy',
                        optimizer='adam',
                        metrics=['accuracy'])

In [21]:
ensemble[0].summary()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_40 (Conv2D)          (None, 128, 128, 16)      448       
                                                                 
 average_pooling2d_10 (Avera  (None, 64, 64, 16)       0         
 gePooling2D)                                                    
                                                                 
 conv2d_41 (Conv2D)          (None, 64, 64, 32)        4640      
                                                                 
 max_pooling2d_30 (MaxPoolin  (None, 32, 32, 32)       0         
 g2D)                                                            
                                                                 
 conv2d_42 (Conv2D)          (None, 32, 32, 64)        18496     
                                                                 
 max_pooling2d_31 (MaxPoolin  (None, 16, 16, 64)     

In [5]:
# Defining Augmentation functions for each class
def shuffling(X):
    X= shuffle(X, random_state=42)
    return X
out_path = 'augmented/'
# Performing Augmentations Class-Wise
# AKIEC
def augment_akiec(image_data):
    out_data = out_path+'akiec'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST)
    ssr_images = glob('augmented/akiec/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/akiec/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/akiec/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    # Selecting remaining images by shuffling all and selecting required
    bc_images = glob('augmented/akiec/*')
    # shuffle and take only required number of images to perform next augmentation:
    bc_images_shuf = shuffling(bc_images)
    target = 4693
    available = len(bc_images)
    bc_selected = bc_images_shuf[0:target-available]
    shiftscalerotate(bc_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=0,
                     shift=0,
                     scale=0.3,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)

#---------------------------------------------------------------------------------------------
    # DF
def augment_df(image_data):
    out_data = out_path+'df'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/df/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/df/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/df/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    bc_images = glob('augmented/df/*')
    shiftscalerotate(bc_images,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    r_images = glob('augmented/df/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=3)
    
#---------------------------------------------------------------------------------------------
# VASC
def augment_vasc(image_data):
    out_data = out_path+'vasc'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/vasc/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/vasc/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    df_images = glob('augmented/vasc/*')
    brightness_contrast(df_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    bc_images = glob('augmented/vasc/*')
    shiftscalerotate(bc_images,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    r_images = glob('augmented/vasc/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=3)

#---------------------------------------------------------------------------------------------
# BCC
def augment_bcc(image_data):
    out_data = out_path+'bcc'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/bcc/*')
    horizontalflip(ssr_images, out_data,
                       augment = True,
                       resize = [128,128],
                       random=1.0)
    hf_images = glob('augmented/bcc/*')
    verticalflip(hf_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    r_images = glob('augmented/bcc/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)

#---------------------------------------------------------------------------------------------
# BKL
def augment_bkl(image_data):
    out_data = out_path+'bkl'
    shiftscalerotate(image_data,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=40,
                     shift=0.0625,
                     scale=0.2,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    ssr_images = glob('augmented/bkl/*')
    verticalflip(ssr_images,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    r_images = glob('augmented/bkl/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=2)
    
#---------------------------------------------------------------------------------------------
# MEL
def augment_mel(image_data):
    out_data = out_path+'mel'
    verticalflip(image_data,
                     out_data,
                     augment = True,
                     resize=[128,128],
                     random=1.0)
    vf_images = glob('augmented/mel/*')
    brightness_contrast(vf_images,
                            out_data,
                            augment = True,
                            resize=[128,128],
                            random=1.0,
                            brightness=0.2, 
                            contrast=0.2,
                            by_max=False)
    r_images = glob('augmented/mel/*')
    # shuffle and take only required number of images to perform next augmentation:
    r_images_shuf = shuffling(r_images)
    target = 4693
    available = len(r_images)
    r_selected = r_images_shuf[0:target-available]
    shiftscalerotate(r_selected,
                     out_data,
                     augment = True,
                     resize = [128,128],
                     random=1.0,
                     rotation=80,
                     shift=0.2,
                     scale=1,
                     interpolation=cv2.INTER_NEAREST,
                     call=1)
    
#---------------------------------------------------------------------------------------------
# NV
# Selecting remaining images by shuffling all and selecting required
def augment_nv(image_data):
    for image in image_data:
        shutil.copyfile(image, f'augmented/nv/{image.split("/")[-1]}')

In [6]:
# Augment, Split and Train
classes = ['akiec','bcc','bkl','df','nv','mel','vasc']

TRAIN_DIR = 'Processed_Data/train'
model_train = [0] * n
epochs = 30
batch_size = 64
for j in range(n):
    # Train test Split
    rs = np.random.randint(100,150)*(j+1)
    for clas in classes:
        os.makedirs(f'augmented/{clas}')
    for cl in classes:
        r_images = glob(f'{TRAIN_DIR}/{cl}/*')
        r_images_shuf = shuffling_tt(r_images,rs)
        train = int(0.7*len(r_images))
        r_train = r_images_shuf[0:train]
        r_test = r_images_shuf[train:]
        if cl == 'akiec':
            augment_akiec(r_train)
        elif cl == 'bcc':
            augment_bcc(r_train)
        elif cl == 'bkl':
            augment_bkl(r_train)
        elif cl == 'df':
            augment_df(r_train)
        elif cl == 'mel':
            augment_mel(r_train)
        elif cl == 'vasc':
            augment_vasc(r_train)
        else:
            augment_nv(r_train)
        if not os.path.isdir(f'test/{cl}'):
            os.makedirs(f'test/{cl}')
        for image in r_test:
            shutil.copyfile(image, f'test/{cl}/{image.split("/")[-1]}')
    # Model Training

    train_dir = 'augmented'
    test_dir = 'test'
    datagen = ImageDataGenerator(rescale = 1./255)

    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical') # set as training data

    validation_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)
    model_train[j] = ensemble[j].fit_generator(train_generator,
                                               steps_per_epoch=train_generator.samples//batch_size,
                                               validation_steps=validation_generator.samples//batch_size,
                                               validation_data = validation_generator,
                                               epochs = epochs)
    print("CNN Model {0:d}: Epochs={1:d}, Training accuracy={2:.5f}, Validation accuracy={3:.5f}".
          format(j+1,epochs,
                 max(model_train[j].history['accuracy']),
                 max(model_train[j].history['val_accuracy']) ))
    if j != 9:
        shutil.rmtree('augmented')
        shutil.rmtree('test')

100%|█████████████████████████████████████| 1525/1525 [00:01<00:00, 1493.75it/s]


Found 32723 images belonging to 7 classes.
Found 5071 images belonging to 7 classes.
Epoch 1/30


2022-03-10 21:42:38.850769: W tensorflow/core/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz
2022-03-10 21:42:39.096068: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.3453 - accuracy: 0.4620

2022-03-10 21:43:16.617396: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


...

Found 32736 images belonging to 7 classes.
Found 3006 images belonging to 7 classes.
Epoch 1/30


2022-03-11 01:42:09.787005: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - ETA: 0s - loss: 1.3378 - accuracy: 0.4708

2022-03-11 01:42:47.745736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


511/511 [==============================] - 51s 99ms/step - loss: 1.3378 - accuracy: 0.4708 - val_loss: 0.9874 - val_accuracy: 0.6253
Epoch 2/30
511/511 [==============================] - 49s 95ms/step - loss: 1.0442 - accuracy: 0.6016 - val_loss: 0.9282 - val_accuracy: 0.6430
Epoch 3/30
511/511 [==============================] - 49s 96ms/step - loss: 0.9101 - accuracy: 0.6576 - val_loss: 0.9005 - val_accuracy: 0.6491
Epoch 4/30
511/511 [==============================] - 50s 98ms/step - loss: 0.8195 - accuracy: 0.6928 - val_loss: 0.8014 - val_accuracy: 0.7086
Epoch 5/30
511/511 [==============================] - 50s 98ms/step - loss: 0.7357 - accuracy: 0.7227 - val_loss: 0.8159 - val_accuracy: 0.6967
Epoch 6/30
511/511 [==============================] - 50s 98ms/step - loss: 0.6671 - accuracy: 0.7517 - val_loss: 0.8213 - val_accuracy: 0.7007
Epoch 7/30
511/511 [==============================] - 49s 96ms/step - loss: 0.6039 - accuracy: 0.7770 - val_loss: 0.9377 - val_accuracy: 0.6634
Epo

In [11]:
#Confution Matrix and Classification Report
#Result
results = np.zeros( (validation_generator.samples,7) ) 
results

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [12]:
# Generating generator for Validation Images
validation_generator = datagen.flow_from_directory(
        test_dir,
        target_size=(128, 128),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

Found 3007 images belonging to 7 classes.


In [13]:
# Predicting classes for vlaidation images
for j in range(n):
    results = results + ensemble[j].predict_generator(validation_generator,
                                                      validation_generator.samples // batch_size+1)

2022-03-11 02:19:21.194461: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:19:33.846620: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:19:46.575704: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:19:59.376326: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:20:12.390119: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:20:25.146918: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-03-11 02:20:37.977862: I tensorflow/core/grappler/optimizers/cust

In [14]:
# Performing Max Voting
results = np.argmax(results,axis = 1)
results

array([0, 0, 0, ..., 6, 6, 6])

In [15]:
print('Confusion Matrix')
print(confusion_matrix(validation_generator.classes, results))
print('Classification Report')
target_names = ['akiec', 'bcc', 'bkl', 'df', 'mel', 'nv', 'vasc']
print(classification_report(validation_generator.classes, results, target_names=target_names))

Confusion Matrix
[[  98    0    0    0    0    1    0]
 [   0  151    0    1    1    2    0]
 [   1    3  314    0    5    6    0]
 [   0    0    0   35    0    0    0]
 [   0    1    2    0  326    5    0]
 [   1    9   21    2   14 1964    1]
 [   0    0    0    0    0    0   43]]
Classification Report
              precision    recall  f1-score   support

       akiec       0.98      0.99      0.98        99
         bcc       0.92      0.97      0.95       155
         bkl       0.93      0.95      0.94       329
          df       0.92      1.00      0.96        35
         mel       0.94      0.98      0.96       334
          nv       0.99      0.98      0.98      2012
        vasc       0.98      1.00      0.99        43

    accuracy                           0.97      3007
   macro avg       0.95      0.98      0.97      3007
weighted avg       0.98      0.97      0.97      3007



In [ ]:
# The End

In [ ]:
# Saving the Model

In [22]:
# Sving Model in Keras Default Format
for k in range(n):
    ensemble[k].save(f'Ensemble2_model{k}')
# Saving Model in H5 Format
for k in range(n):
    ensemble[k].save(f'Ensemble2_model{k}.h5')

2022-03-12 15:41:05.792358: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: Ensemble2_model0/assets
INFO:tensorflow:Assets written to: Ensemble2_model1/assets
INFO:tensorflow:Assets written to: Ensemble2_model2/assets
INFO:tensorflow:Assets written to: Ensemble2_model3/assets
INFO:tensorflow:Assets written to: Ensemble2_model4/assets
INFO:tensorflow:Assets written to: Ensemble2_model5/assets
INFO:tensorflow:Assets written to: Ensemble2_model6/assets
INFO:tensorflow:Assets written to: Ensemble2_model7/assets
INFO:tensorflow:Assets written to: Ensemble2_model8/assets
INFO:tensorflow:Assets written to: Ensemble2_model9/assets


## Loading Saved Model

In [ ]:
# To Load the model
# It can be used to reconstruct the model identically.
# reconstructed_ensemble = [0]*n
# for k in range(n):
#     reconstructed_ensemble[k] = keras.models.load_model(f"Saved Models/Ensemble2/Ensemble2_model{k}")